In [ ]:
import torch
import clip
from PIL import Image
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
print("CLIP model loaded successfully!")
 
IMAGE_FOLDER = "./images"  
image_files = [f for f in os.listdir(IMAGE_FOLDER) if f.endswith(('png', 'jpg', 'jpeg'))]
image_paths = [os.path.join(IMAGE_FOLDER, img) for img in image_files]
images = [preprocess(Image.open(img)).unsqueeze(0).to(device) for img in image_paths]

image_tensors = torch.cat(images)
image_features = model.encode_image(image_tensors)
image_features /= image_features.norm(dim=-1, keepdim=True)

def suggest_images(blog_content, top_n=3):
    """Suggests the top N most relevant images based on blog content."""
    text_tokenized = clip.tokenize([blog_content]).to(device)
    text_features = model.encode_text(text_tokenized)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    
    similarities = (100.0 * text_features @ image_features.T).softmax(dim=-1)
    best_matches = similarities[0].topk(top_n)
    
    return [image_paths[i] for i in best_matches.indices]

blog_content = "A Pretty sunset near ocean with trees."
image_suggestions = suggest_images(blog_content)

print("Suggested Image Paths:", image_suggestions) 


CLIP model loaded successfully!
Suggested Image Paths: ['./images\\4.jpg', './images\\5.jpg', './images\\1.jpg']
